<a href="https://colab.research.google.com/github/hillwalking/2025-2-Data-Analysis/blob/main/ecomileage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [242]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (7,280 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126675 files and dire

In [284]:
# @title 서울시 에코마일리지 에너지 사용량 개인유형

import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 시각화 패키지와 한글 폰트 지정
plt.rc('font', family = 'NanumBarunGothic')

# pandas display option 조정
pd.set_option("display.max_rows", 10)
pd.set_option('display.max_colwidth',1000)
np.set_printoptions(precision=1, suppress=False)

url = 'http://openapi.seoul.go.kr:8088/6f546371447a6e6f3531544a726b62/json/energyUseDataSummaryInfo/1/1000'

response = requests.get(url)

if response.status_code == 200:
    print('API 호출 성공')
    # json data 저장
    data = response.json()
    # data 확인- 1
    print(type(data), ' , data length :  ', len(data))

else:
    print(f'API 호출 실패 : {response.status_code}')
#data

# json data 딕셔녀리 형태 확인{'energyUseDataSummaryInfo': {'list_total_count', 'RESULT', 'row' }
# 필요한 row 만 추출 저장함
data = data['energyUseDataSummaryInfo']['row']

# json data를 파일로 저장하기
with open('ecoSeoul.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii = False)

# 데이터프레임으로 변환
df = pd.DataFrame(data)
#df.info()
#print('\n---DataFrame -----\n',df.head(3))

# 필요한 컬럼만 선택
df = df.loc[:,['YEAR','MON','EUS','GUS','WUS','HUS','MM_TYPE']]

# 개인 유형 데이터 추출한 후, 'MM_TYPE' 삭제
df_p = df.loc[df['MM_TYPE']=='개인']
df_p = df_p.drop(['MM_TYPE'], axis=1)

# 데이터를 숫자형으로 변환하기
df_p['YEAR'] = df_p['YEAR'].apply(lambda  x: int(x))
df_p['MON'] = df_p['MON'].apply(lambda  x: int(x))
df_p['EUS'] = df_p['EUS'].apply(lambda  x: float(x))
df_p['GUS'] = df_p['GUS'].apply(lambda  x: float(x))
df_p['WUS'] = df_p['WUS'].apply(lambda  x: float(x))
df_p['HUS'] = df_p['HUS'].apply(lambda  x: float(x))

# 2015년부터 2024년까지 행 추출
df_p = df_p[(df_p['YEAR'] > 2014) & (df_p['YEAR'] < 2025)]

# 인덱스 정리
df_p = df_p.reset_index(drop=True)

#print('\n----  2-1 ----\n')
#print(df.head(3))

# 월 범위에 따라 계절 지정하기
df_p.loc[(df_p['MON'] ==3) | (df_p['MON'] ==4) | (df_p['MON'] ==5), 'SEASON'] = '봄'
df_p.loc[(df_p['MON'] ==6) | (df_p['MON'] ==7) | (df_p['MON'] ==8), 'SEASON'] = '여름'
df_p.loc[(df_p['MON'] ==9) | (df_p['MON'] ==10) | (df_p['MON'] ==11), 'SEASON'] = '가을'
df_p.loc[(df_p['MON'] ==1) | (df_p['MON'] ==2) | (df_p['MON'] ==12), 'SEASON'] = '겨울'

#컬럼명 한글로 바꾸기
df_p= df_p.rename(columns={'YEAR':'연도', 'MON':'월', 'EUS':'전기', 'GUS':'가스', 'WUS':'수도', 'HUS':'지역난방', 'SEASON':'계절'})
print('\n----  2-2 -----\n',df_p.head(3))

# 계산할 컬럼으로 부분 데이터프레임으로 만든다.
df_sub = df_p.loc[:,['연도','전기', '가스', '수도', '지역난방']]
df_sub

API 호출 성공
<class 'dict'>  , data length :   1

----  2-2 -----
    연도    월   전기        가스       수도       지역난방   계절 
0  2024  12  134.828M  42.103M   7.327M  1.513M  겨울
1  2024  11  257.928M  17.418M  14.679M  8.836M  가을
2  2024  10  282.268M   8.233M  15.298M  2.695M  가을


,연도,전기,가스,수도,지역난방
0,2024,134.828M,42.103M,7.327M,1.513M
1,2024,257.928M,17.418M,14.679M,8.836M
2,2024,282.268M,8.233M,15.298M,2.695M
3,2024,409.873M,5.729M,15.143M,3.421M
4,2024,463.971M,7.072M,15.874M,130.082k
...,...,...,...,...,...
115,2015,177.290M,22.061M,13.087M,1.321M
116,2015,183.892M,37.307M,12.955M,5.836M
117,2015,175.216M,49.985M,12.655M,13.826M
118,2015,189.974M,56.487M,12.657M,18.793M


In [285]:
#YEAR를 기준으로 groupby 해서 sum 하면 연도별 합산된 데이터프레임 생성
df_year = df_sub.groupby('연도').sum()
df_year.head()


,전기,가스,수도,지역난방
연도,,,,
2015,2.308G,330.024M,162.391M,99.762M
2016,2.625G,369.293M,163.944M,102.050M
2017,2.725G,408.997M,162.914M,112.019M
2018,2.943G,459.104M,163.348M,116.432M
2019,2.983G,443.095M,168.844M,195.466M


In [286]:



# 그룹화된 데이터를 sum(axis=1)로 행계산 후 합계열로 추가
df_year['합계'] = df_year.sum(axis=1)
df_year


,전기,가스,수도,지역난방,합계
연도,,,,,
2015,2.308G,330.024M,162.391M,99.762M,2.900G
2016,2.625G,369.293M,163.944M,102.050M,3.260G
2017,2.725G,408.997M,162.914M,112.019M,3.409G
2018,2.943G,459.104M,163.348M,116.432M,3.682G
2019,2.983G,443.095M,168.844M,195.466M,3.791G
2020,3.148G,441.774M,177.549M,139.011M,3.906G
2021,3.358G,444.531M,179.909M,123.191M,4.105G
2022,3.354G,462.833M,176.284M,139.910M,4.133G
2023,3.401G,415.372M,174.086M,130.412M,4.121G
